In [161]:
import keras 
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from pathlib import Path
from keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from keras.layers import LSTM,Dense, Flatten
from keras.models import save_model, load_model
from keras.preprocessing import image
import numpy as np

In [133]:

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [134]:
#Loading the Data Set 
(x_train , y_train), (x_test , y_test) = cifar10.load_data()

In [135]:
#Normalize the data set in the range 0-1 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # ---- As the values are between 0-255 we divide to get values from 0 -1 
x_test  /= 255



In [137]:


# Convert the vectors of training and test results to class matrices as the cifar 10 dataset has 10 classes so each of our vector
#value will denote the weight of each node given to each class
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)


In [138]:
#create the model and add layers 
model = Sequential()

In [139]:
model.add(Conv2D(32,(3,3), padding = 'same' , input_shape = (32,32,3,), activation = "relu" ))
model.add(Conv2D(32,(3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


In [140]:
model.add(Conv2D(64,(3,3), padding = 'same' , activation = "relu" ))
model.add(Conv2D(64,(3,3), activation = "relu" ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [141]:

model.add(Flatten())
model.add(Dense(512,activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10 ,activation = "softmax"))

In [142]:
model.compile (loss = "categorical_crossentropy" , optimizer = "adam", metrics = "accuracy")

In [143]:
model.fit(x_train , y_train , batch_size = 32, epochs = 30, validation_data = (x_test, y_test), shuffle = True)

Epoch 1/30
 140/1563 [=>............................] - ETA: 2:51 - loss: 2.1530 - accuracy: 0.1757

KeyboardInterrupt: 

In [ ]:
model_structure = model.to_json()
f= Path("model_structure.json")
f.write_text(model_structure)
# Creates a HDF5 file 'my_model.h5' 
save_model(model, "model_weights.h5")

In [210]:
class_labels = ["Plane" , "Car", "Bird" , "Cat", "Deer","Dog", "Frog","Horse","Boat","Truck"]

In [211]:
f= Path("model_structure.json")
model_structure = f.read_text()
model = model_from_json(model_structure)


In [212]:
model.load_weights("model_weights.h5")

In [213]:
img = image.load_img("deer.png", target_size = (32,32))

In [214]:
image_to_test = image.img_to_array(img) /255
list_of_images = np.expand_dims(image_to_test, axis= 0 )
results = model.predict(list_of_images)

In [215]:
single_result = results [0]

In [216]:
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

class_label = class_labels[most_likely_class_index]

In [217]:
print("This image is a {} - Likelihood : {:2f}".format (class_label , class_likelihood))

This image is a Deer - Likelihood : 0.971804
